In [5]:
#import necessary modules
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import time

In [6]:
class StockScraper:
    def __init__(self,driver,timeout=10):
        self.driver=driver
        self.wait=WebDriverWait(self.driver,timeout=timeout)
        self.data=[]
    
    
    #Check if webpage is fully loaded
    def wait_for_page_to_load(self):
        page_title=self.driver.title
        try:
            self.wait.until(
                lambda d: d.execute_script("return document.readyState")=="complete"
            )
        except:
            print("The page  \"{page_title}\" did not get fully loaded within the given duration.\n")
        else:
            print(f"The page \"{page_title}\"is successfully loaded.\n")

    def access_url(self,url):
        self.driver.get(url)
        self.wait_for_page_to_load()

    def access_most_active_stocks(self):
        action=ActionChains(self.driver)
        markets_menu=self.wait.until(
            EC.presence_of_element_located((By.XPATH,'/html[1]/body[1]/div[2]/header[1]/div[1]/div[1]/div[1]/div[4]/div[1]/div[1]/ul[1]/li[3]/a[1]'))
        )
        action.move_to_element(markets_menu).perform()

        #click on the Trending Tickers
        trending_tickers=self.wait.until(
            EC.element_to_be_clickable((By.XPATH,'/html[1]/body[1]/div[2]/header[1]/div[1]/div[1]/div[1]/div[4]/div[1]/div[1]/ul[1]/li[3]/div[1]/ul[1]/li[4]/a[1]/div[1]'))
        )
        trending_tickers.click()
        self.wait_for_page_to_load()

        #click on most active
        most_active=self.wait.until(
            EC.element_to_be_clickable((By.XPATH,'/html[1]/body[1]/div[2]/main[1]/section[1]/section[1]/section[1]/section[1]/section[1]/div[1]/nav[1]/ul[1]/li[1]/a[1]/span[1]'))
        )
        most_active.click()
        self.wait_for_page_to_load()

In [7]:
if __name__=="__main__":
    driver=webdriver.Chrome()
    driver.maximize_window()
    url="https://finance.yahoo.com/"
    scraper=StockScraper(driver,5)
    scraper.access_url(url)
    

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=55423): Read timed out. (read timeout=120)